<a href="https://colab.research.google.com/github/CE334/CE334_230586_Kulshreshth_Chikara/blob/main/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Gravimetry Network Adjustment with Drift
# Batch Data: CG-6_0251_G5-CE334.csv
# Course Assignment Submission

In [2]:
import numpy as np
import pandas as pd

In [23]:
# 1. Load CSV Data
file_path = "data.csv"   # Upload this file to Colab first
df = pd.read_csv(file_path)

print("Total Observations:", len(df))
print(df.head())


Total Observations: 30
  Station        Date      Time   CorrGrav  Line  StdDev  StdErr    RawGrav  \
0     FF1  2006-06-14  09:58:05  2938.1049     0  0.0122  0.0022  2919.8542   
1     FF1  2006-06-14  09:58:35  2938.1060     0  0.0110  0.0020  2919.8561   
2     FF1  2006-06-14  09:59:05  2938.1043     0  0.0111  0.0020  2919.8545   
3     TF1  2006-06-14  10:04:28  2936.0332     0  0.0114  0.0021  2917.7870   
4     TF1  2006-06-14  10:04:58  2936.0323     0  0.0124  0.0023  2917.7858   

     X    Y  ...  DriftCorr  MeasurDur  InstrHeight    LatUser    LonUser  \
0 -5.3  4.4  ...    18.3922         30            0  26.515766  80.230446   
1 -3.1  4.9  ...    18.3922         30            0  26.515766  80.230446   
2 -4.5  5.1  ...    18.3922         30            0  26.515766  80.230446   
3 -5.2  4.6  ...    18.3922         30            0  26.515877  80.230370   
4 -7.2  5.3  ...    18.3922         30            0  26.515877  80.230370   

   ElevUser     LatGPS     LonGPS  Elev

In [28]:
# TASK 2: Observation Differencing (Equation 2)

# Convert Time to Decimal Hours
def time_to_decimal(t):
    h, m, s = map(int, t.split(":"))
    return h + m/60 + s/3600

# Apply conversion
df["t_decimal"] = df["Time"].apply(time_to_decimal)

# Reference observation (first measurement)
y1 = df["CorrGrav"].iloc[0]
t1 = df["t_decimal"].iloc[0]

# Compute differenced values
df["delta_y"] = df["CorrGrav"] - y1
df["delta_t"] = df["t_decimal"] - t1

print("Differenced Observations Created:")
df[["Station", "delta_y", "delta_t"]].head()


Differenced Observations Created:


,Station,delta_y,delta_t
0,FF1,0.0000,0.000000
1,FF1,0.0011,0.008333
2,FF1,-0.0006,0.016667
3,TF1,-2.0717,0.106389
4,TF1,-2.0726,0.114722


In [29]:
# TASK 3: Weight Matrix Construction

sigma = df["StdDev"].values

# Weight matrix
W = np.diag(1 / sigma**2)

print("Weight Matrix Shape:", W.shape)
print("\nFirst 5 Weights:")
print(np.diag(W)[:5])


Weight Matrix Shape: (30, 30)

First 5 Weights:
[6718.6240258  8264.46280992 8116.22433244 7694.6752847  6503.64203954]


In [30]:
# TASK 4: Weighted Least Squares Adjustment

# Unique stations
stations = df["Station"].unique()
m = len(stations)

print("Stations:", stations)
print("Total Stations (m):", m)

# Observations count
n = len(df)

# Unknown parameters:
# Δg2...Δgm + drift d
# Total parameters u = m
station_index = {st: i for i, st in enumerate(stations)}

# Build Design Matrix A
A = np.zeros((n, m))

for k in range(n):
    st = df["Station"].iloc[k]
    dt = df["delta_t"].iloc[k]

    idx = station_index[st]
    A[k, idx] = 1          # gravity difference
    A[k, -1] = dt          # drift term

# Datum constraint: fix first station → remove its column
A = A[:, 1:]

# Updated parameter count
u = A.shape[1]

# Observation vector
l = df["delta_y"].values.reshape(-1, 1)

# Solve Normal Equation
N = A.T @ W @ A
U = A.T @ W @ l

x_hat = np.linalg.inv(N) @ U

print("\nEstimated Parameters Vector:")
print(x_hat)

# Redundancy
r = n - u

print("\nNumber of Observations (n):", n)
print("Number of Parameters (u):", u)
print("Redundancy (r):", r)


Stations: ['FF1' 'TF1' 'FF2' 'TF2' '5F1' 'TF3' '5F2' '7F1' '5F3' '7F2']
Total Stations (m): 10

Estimated Parameters Vector:
[[-1.16213647]
 [ 1.64203866]
 [ 0.42359306]
 [-1.12630055]
 [ 1.69912515]
 [ 0.14829867]
 [-1.31765561]
 [ 1.51238974]
 [-8.03527363]]

Number of Observations (n): 30
Number of Parameters (u): 9
Redundancy (r): 21


In [31]:
# TASK 4: Variance Computation

# Residuals
v = A @ x_hat - l

# A priori variance
sigma0_apriori = 1

# A posteriori variance
sigma0_sq = (v.T @ W @ v) / r

print("A Priori Variance σ0² =", sigma0_apriori)
print("A Posteriori Variance σ̂0² =", float(sigma0_sq))


A Priori Variance σ0² = 1
A Posteriori Variance σ̂0² = 22.990514728441923


/tmp/ipython-input-635419626.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("A Posteriori Variance σ̂0² =", float(sigma0_sq))


In [32]:
# TASK 5: Variance–Covariance Matrix

Cov_x = sigma0_sq * np.linalg.inv(N)

print("\nVariance–Covariance Matrix:")
print(Cov_x)

# Standard deviation of parameters
std_params = np.sqrt(np.diag(Cov_x))

print("\nStandard Deviations of Estimated Parameters:")
print(std_params)



Variance–Covariance Matrix:
[[ 0.00130381  0.0001214   0.00018386  0.00023135  0.00027851  0.00032572
   0.00037821  0.00042704 -0.00058903]
 [ 0.0001214   0.00135436  0.00033465  0.00042107  0.00050692  0.00059285
   0.00068838  0.00077726 -0.00107209]
 [ 0.00018386  0.00033465  0.00324031  0.00063772  0.00076773  0.00089786
   0.00104255  0.00117715 -0.00162367]
 [ 0.00023135  0.00042107  0.00063772  0.00301758  0.00096601  0.00112975
   0.00131181  0.00148118 -0.00204302]
 [ 0.00027851  0.00050692  0.00076773  0.00096601  0.00395624  0.00136008
   0.00157924  0.00178314 -0.00245952]
 [ 0.00032572  0.00059285  0.00089786  0.00112975  0.00136008  0.00396914
   0.00184694  0.0020854  -0.00287644]
 [ 0.00037821  0.00068838  0.00104255  0.00131181  0.00157924  0.00184694
   0.00462375  0.00242145 -0.00333996]
 [ 0.00042704  0.00077726  0.00117715  0.00148118  0.00178314  0.0020854
   0.00242145  0.00532334 -0.00377118]
 [-0.00058903 -0.00107209 -0.00162367 -0.00204302 -0.00245952 -0.002

In [39]:
# Correct Design Matrix A

# -----------------------------
# Recompute Adjustment Solution
# -----------------------------

# Observation vector
l = df["delta_y"].values.reshape(-1, 1)

# Weight matrix
sigma = df["StdDev"].values
W = np.diag(1 / sigma**2)

# Normal equation
N = A.T @ W @ A
U = A.T @ W @ l

# Solve for parameters
x_hat = np.linalg.inv(N) @ U

# Residuals
v = A @ x_hat - l

# Redundancy
n_obs = len(df)
u_params = A.shape[1]
r = n_obs - u_params

# Posterior variance
sigma0_sq = (v.T @ W @ v) / r

# Covariance matrix
Cov_x = sigma0_sq * np.linalg.inv(N)

# Standard deviations
std_params = np.sqrt(np.diag(Cov_x))

print("x_hat shape:", x_hat.shape)
print("std_params shape:", std_params.shape)
param_names = list(stations[1:]) + ["Drift d"]

print("\n===============================")
print("FINAL ADJUSTED PARAMETERS")
print("===============================")

for i, name in enumerate(param_names):
    print(f"{name:6s} = {x_hat[i,0]:10.6f} ± {std_params[i]:.6f}")



x_hat shape: (10, 1)
std_params shape: (10,)

FINAL ADJUSTED PARAMETERS
TF1    =  -2.066227 ± 0.003148
FF2    =  -0.003489 ± 0.005665
TF2    =  -2.068555 ± 0.008582
5F1    =  -4.262093 ± 0.010775
TF3    =  -2.075957 ± 0.012967
5F2    =  -4.266699 ± 0.015154
7F1    =  -6.444107 ± 0.017591
5F3    =  -4.275934 ± 0.019858
7F2    =  -6.449315 ± 0.022089
Drift d =  -0.051320 ± 0.027369
